In [174]:
import numpy as np
import pandas as pd
import nltk
import re
import PyPDF2
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.tokenize import sent_tokenize 
import collections
from tika import parser
import csv
import sys
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
import io



path = (r'/Users/rajsharavan/Desktop/Python/Splitted PDF/Sample44.pdf')
# pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

# pageNumbers = pdfReader.numPages

# print(pageNumbers)

# textList = []

# for i in range(pageNumbers):
#     pageObj = pdfReader.getPage(i)
#     text = pageObj.extractText()
#     #print(text)
#     textList.append(text)

# print(textList[1])

#This Function converts the Merged PDF to text
def pdfparser(data):

    fp = open(data,'rb')
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
# Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
# Process each page contained in the document.
    for page in PDFPage.get_pages(fp):
        interpreter.process_page(page)
        data =  retstr.getvalue()
    #print(data)
    return data

# pdfparser(path)
parsedText = pdfparser(path)
#print(parsedText)

In [175]:
#Cleaning the text
import string
filtered_sentence = []
def clean(parsedText):
    #stop_words = set(stopwords.words('english'))   
    word_tokens = word_tokenize(parsedText)
    words = [w for w in word_tokens if w.isalpha()]
    for w in words:
        filtered_sentence.append(w.lower())
         #if w.lower() not in stop_words:          
    return filtered_sentence

cleanedText = clean(parsedText)
#print(cleanedText)
# processedText = word_tokenize(cleanedText)

# print (processedText)

In [176]:
#POS_Tagging for Lemmatization
POS_tag = nltk.pos_tag(cleanedText)
#print(POS_tag)

In [177]:
#Lemmatization
#nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

wordnetLemmatizer = WordNetLemmatizer()

adjectiveTags = ['JJ','JJR','JJS']

lemmatizedText = []

for word in POS_tag:
    if word[1] in adjectiveTags:
        lemmatizedText.append(str(wordnetLemmatizer.lemmatize(word[0],pos="a")))
    else:
        lemmatizedText.append(str(wordnetLemmatizer.lemmatize(word[0])))
        
#print(lemmatizedText)

In [178]:
#POS tagging for Filtering

POS_tag = nltk.pos_tag(lemmatizedText)

#print "Lemmatized text with POS tags: \n"
#print (POS_tag)

In [179]:
#POS Based Filtering

stopwords = []

wantedPOS = ['NN','NNS','NNP','JJ','JJR','JJS','VBG','FW']

for word in POS_tag:
    if word[1] not in wantedPOS:
        stopwords.append(word[0])

punctuations = list(str(string.punctuation))

stopwords = stopwords+punctuations

stopword_file = open(r'/Users/rajsharavan/Desktop/Python/long_stopwords.txt', 'r')
#Source = https://www.ranks.nl/stopwords

lots_of_stopwords = []

for line in stopword_file.readlines():
    lots_of_stopwords.append(str(line.strip()))

stopwords_plus = []
stopwords_plus = stopwords + lots_of_stopwords
stopwords_plus = set(stopwords_plus)

In [180]:
#Removing Stopwords
processed_text = []
for word in lemmatizedText:
    if word not in stopwords_plus:
        processed_text.append(word)
#print(processed_text)

In [181]:
#Vocabulary Creation which will contain only unique words from processed text
vocabulary = list(set(processed_text))
#print(vocabulary)
print(len(vocabulary))

1216


In [182]:
#Building a graph [ This is similar to PageRank Algorithm]
import math
vocab_len = len(vocabulary)

#All the words in vocabulary list is called as vertices (singular: vertex)
weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

score = np.zeros((vocab_len),dtype=np.float32) #returns an array of given shape and type filled with zeroes
window_size = 3
covered_coocurrences = [] #List to hold the weighted edge of each vertices

for i in range(0,vocab_len):
    score[i]=1
    for j in range(0,vocab_len):
        if j==i: #No link between the voccabulary and hence its propbability will be initialized to 0
            weighted_edge[i][j]=0
        else:
            for window_start in range(0,(len(processed_text)-window_size)):
                
                window_end = window_start+window_size
                
                window = processed_text[window_start:window_end]
                
                if (vocabulary[i] in window) and (vocabulary[j] in window):
                    
                    index_of_i = window_start + window.index(vocabulary[i]) #Finding the indexes inorder to compare with the existing words within the co-occurences list
                    index_of_j = window_start + window.index(vocabulary[j])
                    
                    # index_of_x is the absolute position of the xth term in the window 
                    # (counting from 0) 
                    # in the processed_text
                      
                    if [index_of_i,index_of_j] not in covered_coocurrences:
                        weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j) #Calculating weights between ecah vertices by formula (1/difference between positions of two vertices) 
                        covered_coocurrences.append([index_of_i,index_of_j]) #math.fabs gives the absolute value
                        
#Calculating the weighted summation of connections of a vertex

inout = np.zeros((vocab_len),dtype=np.float32)

for i in range(0,vocab_len):
    for j in range(0,vocab_len):
        inout[i]+=weighted_edge[i][j]  #sum of each edge connections of words at i and j

#Scoring Vertices

#The formula used for scoring a vertex represented by i is:
    #Score[i] = (1-d) * Summation(j)
    #j belongs to list of each vertices that has a connection with i
    #summation of j means  Sum of (weighted_edge[i][j]/inout[j])*score[j]
    
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(0,vocab_len):
        
        summation = 0
        for j in range(0,vocab_len):
            if weighted_edge[i][j] != 0:
                summation += (weighted_edge[i][j]/inout[j])*score[j]
                
        score[i] = (1-d) + d*(summation)
    
    if np.sum(np.fabs(prev_score-score)) <= threshold:
        #convergence condition---- This means where the score is almost equal to zero i.e a threshold condition
        print ("Converging at iteration "+str(iter)+"....")
        break

import csv
import collections

def check():
    filtered_sentence1 = []
    for i in range(0,vocab_len):
        test1=(vocabulary[i], str(score[i]))
        filtered_sentence1.append(test1)
    #print(filtered_sentence1)
        #print(type(test1))
        #print('Score of',vocabulary[i],':', str(score[i]))
    def get_count(word_count_tuple):
        return word_count_tuple[1]

    word_counts1 = collections.Counter(filtered_sentence1)

# This function sorts the words based on the number of times repeated and saves them in a csv file
    def print_top():
        word_count1 = word_counts1
        items = sorted(word_count1.items(),key=get_count,reverse=True)
        with open('/Users/rajsharavan/Desktop/Python/Splitted PDF/Textrank/TextRank_Sample44.csv','w') as out:
            csv_out=csv.writer(out)
            #csv_out.writerow('Name')
            for row in items:
                csv_out.writerow(row)

    print_top()  
check()

Converging at iteration 36....
